## The api for Facebook & Instagram don't support to serach keyword，only can search page related information；By now, only Twitter and Reddit support to search keyword.

# Reddit scraper (based on PRAW)

## Achieved functions:  
    1. Get subreddit data: Grab data from specific subreddit, like Top10 hot posts in 'wallstreetbets'.
    2. Scrape the key word related posts and comments
    3. Scrape the relevant posts and comments (with specific stock symbols)
    

### 🌟Attention: It can not directly search for the comments in the Reddit. Therefore, I search for the posts and post related comments here.

## 1. Install praw and create a reddit app

In [16]:
!pip install praw

     |████████████████████████████████| 167 kB 5.4 MB/s eta 0:00:01
     |████████████████████████████████| 52 kB 8.4 MB/s  eta 0:00:01


In [17]:
import praw
reddit = praw.Reddit(client_id='I-kcAsDBso9JnjaQTzQaSw', client_secret='UuBjsa6O7LNTLnF0nXHeUnWKD9aefQ', user_agent='Scrapping Test')

## 2. Get subreddit data

In [18]:
hot_posts = reddit.subreddit('all').hot(limit=10) # Scrape the Top 10 hot posts from all the subreddit
for post in hot_posts:
    print(post.title)

Grandpa cries when he discovers that the baby is named after him
SLPT: How to end poverty
Poor guy. Went from stoked to I hate myself real quick.
Metric system vs Imperial system.
Knees to the face.
Oh lawd he coming!
🔥 Alligator seen (up close) from paddleboard
Tesla opens a showroom on Native American land in New Mexico, getting around the state's ban on automakers selling vehicles straight to consumers
Stupidity at its finest
I built a PC inside an RC Semi Truck!


In [44]:
import pandas as pd
## Scrape the Top100 hot posts from 'wallstreetbets' subreddit, and save them information into dataframe

posts = []
subreddit = reddit.subreddit('wallstreetbets') 
for post in subreddit.hot(limit=100):
    posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])
print(posts)

                                                title  score      id  \
0    What Are Your Moves Tomorrow, September 14, 2021     38  pnn4wa   
1   Daily Popular Tickers Thread for September 13,...    241  pndzcb   
2                     It’s a lot worse than you think   9681  pnelsm   
3                                       What is loss?  12395  pn8akm   
4   2 years ago, I Yolo’d my entire life savings i...    244  pnnmzn   
..                                                ...    ...     ...   
95  Epic Games appeals ruling that Apple’s App Sto...     53  pn1c9m   
96           Pt. II. Degenerates, We’re Going Nuclear    278  pmrg6i   
97  $SLQT Updated DD: 72% increase as predicated i...     31  pn38ld   
98                              This is a wakeup call   3199  pmgszo   
99     Wife is shaming me for not making enough gains   1122  pml2zl   

         subreddit                                                url  \
0   wallstreetbets  https://www.reddit.com/r/wallstreetbets/co

In [91]:
posts['url'][1]

'https://www.reddit.com/r/AskReddit/comments/pk1ju3/whats_a_job_that_you_just_associate_with_jerks/'

### Try scrape all the comments with a specific post url

In [88]:
# Title: 'Daily Popular Tickers Thread for September 13, 2021 - GME | CLOV | AMC'
url = "https://www.reddit.com/r/wallstreetbets/comments/pndzcb/daily_popular_tickers_thread_for_september_13/"
submission = reddit.submission(url=url)

from praw.models import MoreComments
import datetime


comments = []
for comment in submission.comments:
    if isinstance(comment, MoreComments):
        continue
    if comment.body == 'NaN': # Ignore the null value 
        continue
    create_time = datetime.datetime.fromtimestamp(comment.created_utc) # Convert the Unixtimestamp into date time 
    comments.append([comment.author, comment.body, comment.score, comment.id, create_time, comment.permalink, comment.subreddit])

comments = pd.DataFrame(comments,columns=['author', 'comment', 'score', 'id', 'create time', 'permalink', 'subreddit'])
comments

,author,comment,score,id,create time,permalink,subreddit
0,Premier_Legacy,Do people actually hold GME long term here ? I...,47,hcpxt9z,2021-09-13 14:20:21,/r/wallstreetbets/comments/pndzcb/daily_popula...,wallstreetbets
1,Aether_wolf,Wtf CLOV? It was down 3% when I checked it a h...,40,hcp5tgu,2021-09-13 11:07:59,/r/wallstreetbets/comments/pndzcb/daily_popula...,wallstreetbets
2,noicedeb8r,"Come on CLOV, slow and steady",35,hcp4s3r,2021-09-13 11:00:47,/r/wallstreetbets/comments/pndzcb/daily_popula...,wallstreetbets
3,Hairy_CoinPurse,Gamestop to 250 by end of week or I delete my ...,35,hcp7kvr,2021-09-13 11:20:11,/r/wallstreetbets/comments/pndzcb/daily_popula...,wallstreetbets
4,collectorkabbash,$GME 😁,30,hcpx41n,2021-09-13 14:15:34,/r/wallstreetbets/comments/pndzcb/daily_popula...,wallstreetbets
...,...,...,...,...,...,...,...
244,One_of_the_blessings,CLOV sucks,-9,hcp841w,2021-09-13 11:23:49,/r/wallstreetbets/comments/pndzcb/daily_popula...,wallstreetbets
245,mediumrarestonks,Infinite money glitch is AMC $65/$70 one month...,-1,hcqd826,2021-09-13 16:04:16,/r/wallstreetbets/comments/pndzcb/daily_popula...,wallstreetbets
246,Stonk_isthe_way,Time to short CLOV to the core 🤑,-12,hcojqwi,2021-09-13 08:11:20,/r/wallstreetbets/comments/pndzcb/daily_popula...,wallstreetbets
247,TheSouthAlwaysFails,Clov fucked your wives while it made you sit i...,-16,hcp9w0e,2021-09-13 11:36:03,/r/wallstreetbets/comments/pndzcb/daily_popula...,wallstreetbets


In [89]:
comments['comment'][0]

'Do people actually hold GME long term here ? If so, that’s dope'

## 3. Scrape the key word related posts and comments

`search(query: str, sort: str = 'relevance', syntax: str = 'lucene', time_filter: str = 'all', **generator_kwargs: Any)→ Iterator[praw.models.Submission]`    

Return a ListingGenerator for items that match query.

Parameters
    
    query – The query string to search for.

    sort – Can be one of: relevance, hot, top, new, comments. (default: relevance).

    syntax – Can be one of: cloudsearch, lucene, plain (default: lucene).

    time_filter – Can be one of: all, day, hour, month, week, year (default: all).


For more information on building a search query see: https://www.reddit.com/wiki/search

For example, to search all subreddits for praw try:

`for submission in reddit.subreddit("all").search("praw"):
    print(submission.title)
`

In [94]:
submissions = []
for submission in reddit.subreddit("all").search("CLOV", 'relevance', 'lucene', 'week'):
    if submission.title == 'NaN': # Ignore the null value 
        continue
    create_time = datetime.datetime.fromtimestamp(submission.created_utc) # Convert the Unixtimestamp into date time 
    submissions.append([submission.id, create_time, submission.author, submission.subreddit, submission.title, submission.score, submission.upvote_ratio, submission.num_comments, submission.comments, submission.url, submission.permalink])

submissions = pd.DataFrame(comments,columns=['id', 'create time', 'author', 'subreddit', 'title', 'score', 'upvote_ratio', 'comments', 'num_comments', 'url', 'permalink', 'score'])
submissions

,id,create time,author,subreddit,title,score,upvote_ratio,comments,num_comments,url,permalink,score
0,hcpxt9z,2021-09-13 14:20:21,Premier_Legacy,wallstreetbets,NaN,47,NaN,NaN,NaN,NaN,/r/wallstreetbets/comments/pndzcb/daily_popula...,47
1,hcp5tgu,2021-09-13 11:07:59,Aether_wolf,wallstreetbets,NaN,40,NaN,NaN,NaN,NaN,/r/wallstreetbets/comments/pndzcb/daily_popula...,40
2,hcp4s3r,2021-09-13 11:00:47,noicedeb8r,wallstreetbets,NaN,35,NaN,NaN,NaN,NaN,/r/wallstreetbets/comments/pndzcb/daily_popula...,35
3,hcp7kvr,2021-09-13 11:20:11,Hairy_CoinPurse,wallstreetbets,NaN,35,NaN,NaN,NaN,NaN,/r/wallstreetbets/comments/pndzcb/daily_popula...,35
4,hcpx41n,2021-09-13 14:15:34,collectorkabbash,wallstreetbets,NaN,30,NaN,NaN,NaN,NaN,/r/wallstreetbets/comments/pndzcb/daily_popula...,30
...,...,...,...,...,...,...,...,...,...,...,...,...
244,hcp841w,2021-09-13 11:23:49,One_of_the_blessings,wallstreetbets,NaN,-9,NaN,NaN,NaN,NaN,/r/wallstreetbets/comments/pndzcb/daily_popula...,-9
245,hcqd826,2021-09-13 16:04:16,mediumrarestonks,wallstreetbets,NaN,-1,NaN,NaN,NaN,NaN,/r/wallstreetbets/comments/pndzcb/daily_popula...,-1
246,hcojqwi,2021-09-13 08:11:20,Stonk_isthe_way,wallstreetbets,NaN,-12,NaN,NaN,NaN,NaN,/r/wallstreetbets/comments/pndzcb/daily_popula...,-12
247,hcp9w0e,2021-09-13 11:36:03,TheSouthAlwaysFails,wallstreetbets,NaN,-16,NaN,NaN,NaN,NaN,/r/wallstreetbets/comments/pndzcb/daily_popula...,-16


In [ ]:
submissions = []
for submission in reddit.subreddit("all").search("CLOV", 'relevance', 'lucene', 'week'):
    if submission.title == 'NaN': # Ignore the null value 
        continue
    create_time = datetime.datetime.fromtimestamp(submission.created_utc) # Convert the Unixtimestamp into date time 
    submissions.append([submission.id, create_time, submission.author, submission.subreddit, submission.title, submission.score, submission.upvote_ratio, submission.num_comments, submission.comments, submission.url, submission.permalink])

submissions = pd.DataFrame(comments,columns=['id', 'create time', 'author', 'subreddit', 'title', 'score', 'upvote_ratio', 'comments', 'num_comments', 'url', 'permalink', 'score'])
submissions

## 4. Read all the symbols of stock

In [38]:
# Download all the stock information from 'https://www.nasdaq.com/market-activity/stocks/screener'
stock_info = pd.read_csv('nasdaq_screener_1631565856063.csv')

In [39]:
stock_info.head()

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
0,A,Agilent Technologies Inc. Common Stock,$172.08,-5.15,-2.906%,5.209252e+10,United States,1999.0,2251336,Capital Goods,Electrical Products
1,AA,Alcoa Corporation Common Stock,$49.22,-0.28,-0.566%,9.197552e+09,NaN,2016.0,8561926,Basic Industries,Metal Fabrications
2,AAC,Ares Acquisition Corporation Class A Ordinary ...,$9.78,0.02,0.205%,1.222500e+09,NaN,2021.0,563121,Finance,Business Services
3,AACG,ATA Creativity Global American Depositary Shares,$2.87,0.02,0.702%,9.098241e+07,China,NaN,33855,Miscellaneous,Service to the Health Industry
4,AACIU,Armada Acquisition Corp. I Unit,$9.88,0.00,0.00%,0.000000e+00,United States,2021.0,552,NaN,NaN


In [40]:
stock_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8085 entries, 0 to 8084
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Symbol      8085 non-null   object 
 1   Name        8085 non-null   object 
 2   Last Sale   8085 non-null   object 
 3   Net Change  8085 non-null   float64
 4   % Change    8081 non-null   object 
 5   Market Cap  7461 non-null   float64
 6   Country     7319 non-null   object 
 7   IPO Year    4756 non-null   float64
 8   Volume      8085 non-null   int64  
 9   Sector      6145 non-null   object 
 10  Industry    6158 non-null   object 
dtypes: float64(3), int64(1), object(7)
memory usage: 694.9+ KB


In [49]:
symbols = []
for symbol in stock_info['Symbol']:
    symbols.append(symbol)
symbols

['A',
 'AA',
 'AAC',
 'AACG',
 'AACIU',
 'AADI',
 'AAIC',
 'AAIC^B',
 'AAIC^C',
 'AAIN',
 'AAL',
 'AAMC',
 'AAME',
 'AAN',
 'AAOI',
 'AAON',
 'AAP',
 'AAPL',
 'AAQC',
 'AAT',
 'AATC',
 'AAU',
 'AAWW',
 'AB',
 'ABB',
 'ABBV',
 'ABC',
 'ABCB',
 'ABCL',
 'ABCM',
 'ABEO',
 'ABEV',
 'ABG',
 'ABGI',
 'ABIO',
 'ABM',
 'ABMD',
 'ABNB',
 'ABOS',
 'ABR',
 'ABR^D',
 'ABR^E',
 'ABSI',
 'ABST',
 'ABT',
 'ABTX',
 'ABUS',
 'ABVC',
 'AC',
 'ACA',
 'ACAD',
 'ACAH',
 'ACAHU',
 'ACAHW',
 'ACB',
 'ACBA',
 'ACBAW',
 'ACBI',
 'ACC',
 'ACCD',
 'ACCO',
 'ACEL',
 'ACER',
 'ACET',
 'ACEV',
 'ACEVU',
 'ACEVW',
 'ACGL',
 'ACGLN',
 'ACGLO',
 'ACGLP',
 'ACH',
 'ACHC',
 'ACHL',
 'ACHV',
 'ACI',
 'ACIC',
 'ACII',
 'ACIU',
 'ACIW',
 'ACKIT',
 'ACKIU',
 'ACKIW',
 'ACLS',
 'ACM',
 'ACMR',
 'ACN',
 'ACNB',
 'ACOR',
 'ACP',
 'ACP^A',
 'ACQR',
 'ACQRU',
 'ACQRW',
 'ACR',
 'ACR^C',
 'ACR^D',
 'ACRE',
 'ACRO',
 'ACRS',
 'ACRX',
 'ACST',
 'ACTD',
 'ACTDU',
 'ACTDW',
 'ACTG',
 'ACU',
 'ACV',
 'ACVA',
 'ACXP',
 'ACY',
 'ADAG',


### Import the symbols of MeMe Stock

In [96]:
meme_stocks = ['NCTY',  'ZOM',  'GME',  'NAKD',  'CTRM',  'KOSS',  'AMC',  
'EXPR',  'CCIV',  'SNDL',  'JAGX',  'GTE',  'LLIT',  'TRVG',
'STPK',  'BB',  'REPX',  'BBBY',  'IPOE',  'SRNE',  'ACB',
'OPEN',  'INO',  'AG',  'CRIS',  'GM',  'RKT',  'SIEB',
'IPOF',  'MUX',  'HIMS',  'BYDDY',  'TR',  'NOK',  'RYCEY',
'WKHS',  'CLOV']

meme_stocks

['NCTY',
 'ZOM',
 'GME',
 'NAKD',
 'CTRM',
 'KOSS',
 'AMC',
 'EXPR',
 'CCIV',
 'SNDL',
 'JAGX',
 'GTE',
 'LLIT',
 'TRVG',
 'STPK',
 'BB',
 'REPX',
 'BBBY',
 'IPOE',
 'SRNE',
 'ACB',
 'OPEN',
 'INO',
 'AG',
 'CRIS',
 'GM',
 'RKT',
 'SIEB',
 'IPOF',
 'MUX',
 'HIMS',
 'BYDDY',
 'TR',
 'NOK',
 'RYCEY',
 'WKHS',
 'CLOV']

## 5. Scrape the relevant posts and comments (using the symbols above)

### 5.1 Scrape the relevant posts information   

In [ ]:
# Scrape the whole Stocks data (with 8085 stocks)
posts = []
for symbol in symbols:   
    subreddit = reddit.subreddit('all').search(symbol, 'relevance', 'lucene', 'week')
    for post in subreddit:
        posts.append([symbol, post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])

posts = pd.DataFrame(posts,columns=['symbol', 'title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])
posts.to_csv('Reddit posts information.csv')

In [113]:
# Scrape the MeMe Stocks data
posts = []
count = 0
sample = meme_stocks[0:10]
for symbol in sample:   
    subreddit = reddit.subreddit('all').search(symbol, 'relevance', 'lucene', 'week')
    count += 1
    print('Search for ', symbol, '. Have searched', count)
    for post in subreddit:
        create_time = datetime.datetime.fromtimestamp(post.created) # Convert the Unixtimestamp into date time 
        posts.append([symbol, post.id, post.title, create_time, post.score, post.subreddit, post.url, post.num_comments, post.selftext])

posts = pd.DataFrame(posts,columns=['symbol', 'id', 'title', 'create time', 'score', 'subreddit', 'url', 'num_comments', 'body'])
posts.to_csv('Reddit memestock posts information (sample).csv')

Search for  NCTY . Have searched 1
Search for  ZOM . Have searched 2
Search for  GME . Have searched 3
Search for  NAKD . Have searched 4
Search for  CTRM . Have searched 5
Search for  KOSS . Have searched 6
Search for  AMC . Have searched 7
Search for  EXPR . Have searched 8
Search for  CCIV . Have searched 9
Search for  SNDL . Have searched 10


In [116]:
posts['symbol'=='GME']

KeyError: False

In [115]:
posts['body'][604]

'Fri and Tues were very low volume red days. It\'s been in a .7 to .8 consolidation trend for some time now. There are 4 types of trades going on right now. 1. Holders that tend to buy on the Dips rather than the upswing limiting upward momentum 2. Shorts that recognize the pattern and enforce it by shorting at upper .70s and covering at lower 70s 3. Swing traders that are enforcing the pattern by buying in low .70s and selling in high 70s. 4. Investors on the sideline waiting for a break up move to jump in. All these combined just keep it in this pattern. To break out there needs to be a catalyst. 1. Some form of news like an m&a 2. Institutional investors decide they have acquired enough and stop shorting so their main holding will grow value 3. An abnormal "retail driven spike" that causes price to go back into .80s and cause the sideline investors to jump in. End of consolidation is very hard to predict but when ends its usually pretty fast. Worst case if none of these occur in nea

### 5.2 Scrape the relevant comments information